In [1]:
from game import Game
import time
import os
import re

## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

In [10]:
# TODO: Must return moves (if there is no solution return None), number of visited states
from collections import deque
def solver_bfs(game_map):
    game = Game(game_map)
    goal_locations = game.get_goal_locations()
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    fringe = deque([(initial_state, [])])
    explored = set()
    explored.add(initial_state)
    
    while fringe:
        (player_pos, box_positions), path = fringe.popleft()
        
        if all(box_positions[i] == goal_locations[i] for i in range(len(box_positions))):
            return path, len(explored)
        
        for direction in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_pos)
            game.set_box_positions(box_positions)
            
            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))
                if new_state not in explored:
                    explored.add(new_state)
                    fringe.append((new_state, path + [direction]))
    
    return None, len(explored)

### DFS

In [11]:
# TODO: Must return moves, number of visited states
def solver_dfs(game_map):
    game = Game(game_map)
    goal_locations = game.get_goal_locations()
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    fringe = [(initial_state, [])]
    explored = set()
    explored.add(initial_state)
    
    while fringe:
        (player_pos, box_positions), path = fringe.pop()
        
        if all(box_positions[i] == goal_locations[i] for i in range(len(box_positions))):
            return path, len(explored)
        
        for direction in ['R', 'L', 'D', 'U']:
            game.set_player_position(player_pos)
            game.set_box_positions(box_positions)
            
            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))
                if new_state not in explored:
                    explored.add(new_state)
                    fringe.append((new_state, path + [direction]))
    
    return None, len(explored)

### IDS

In [12]:
# TODO: Must return moves, number of visited states
def solver_ids(game_map):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    goal_locations = game.get_goal_locations()

    for depth_limit in range(1, 1000):
        fringe = [(initial_state, [], 0)]
        explored = set([initial_state])

        while fringe:
            (player_pos, box_positions), path, depth = fringe.pop()

            if all(box_positions[i] == goal_locations[i] for i in range(len(box_positions))):
                return path, len(explored)

            if depth < depth_limit:
                for direction in ['R', 'L', 'D', 'U']:
                    game.set_player_position(player_pos)
                    game.set_box_positions(box_positions)

                    if game.apply_move(direction):
                        new_state = (game.get_player_position(), tuple(game.get_box_locations()))
                        if new_state not in explored:
                            explored.add(new_state)
                            fringe.append((new_state, path + [direction], depth + 1))

    return None, len(explored)

### A*

In [13]:
# TODO
def heuristic(player_pos, box_positions, goal_positions):
    total_distance = 0
    min_player_box_distance = float('inf')
    for i in range(len(box_positions)):
        box = box_positions[i]
        goal = goal_positions[i]
        box_goal_distance = abs(box[0] - goal[0]) + abs(box[1] - goal[1])
        total_distance += box_goal_distance
        
        if box != goal:
            player_box_distance = abs(player_pos[0] - box[0]) + abs(player_pos[1] - box[1])
            min_player_box_distance = min(min_player_box_distance, player_box_distance)
    
    if min_player_box_distance == float('inf'):
        min_player_box_distance = 0
    
    total_distance += min_player_box_distance
    return total_distance * 0.47 # 0.47 is for addmissivlity of the heuristic

# TODO: Must return moves, number of explored states
import heapq
def solver_astar(game_map, weight=1):
    game = Game(game_map)
    initial_player_pos = game.get_player_position()
    initial_box_positions = tuple(game.get_box_locations())
    goal_positions = game.get_goal_locations()
    
    initial_state = (initial_player_pos, initial_box_positions)
    initial_h = heuristic(initial_player_pos, initial_box_positions, goal_positions)
    
    counter = 0
    fringe = [(initial_h, counter, 0, initial_state, [])]
    # state -> g-value
    explored = {initial_state: 0}
    
    while fringe:
        _, _, g_value, (player_pos, box_positions), path = heapq.heappop(fringe)
        
        if all(box_positions[i] == goal_positions[i] for i in range(len(box_positions))):
            return path, len(explored)
        
        for direction in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_pos)
            game.set_box_positions(box_positions)
            
            if game.apply_move(direction):
                new_player_pos = game.get_player_position()
                new_box_positions = tuple(game.get_box_locations())
                new_state = (new_player_pos, new_box_positions)
                new_g = g_value + 1
                
                if new_state not in explored or new_g < explored[new_state]:
                    explored[new_state] = new_g
                    
                    h_value = heuristic(new_player_pos, new_box_positions, goal_positions)
                    f_value = new_g + weight * h_value
                    
                    counter += 1
                    heapq.heappush(fringe, (f_value, counter, new_g, new_state, path + [direction]))
    
    return None, len(explored)

## Solve

In [14]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "Weighted A*": lambda game_map: solver_astar(game_map, 6)
}

In [15]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")

In [16]:
def solve_all(map):
    for method in SOLVERS.keys():
        solve(map, method)

#### Map 1:

In [17]:
solve_all(1)

BFS took 0.0 seconds on map 1 and visited 47 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
DFS took 0.0 seconds on map 1 and visited 17 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
IDS took 0.0 seconds on map 1 and visited 17 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
A* took 0.0 seconds on map 1 and visited 47 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
Weighted A* took 0.0 seconds on map 1 and visited 17 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']


#### Map 2:

In [18]:
solve_all(2)

BFS took 0.0 seconds on map 2 and visited 26 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
DFS took 0.0 seconds on map 2 and visited 13 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
IDS took 0.0 seconds on map 2 and visited 13 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
A* took 0.0 seconds on map 2 and visited 26 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
Weighted A* took 0.0 seconds on map 2 and visited 13 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']


#### Map 3:

In [19]:
solve_all(3)

BFS took 0.0 seconds on map 3 and visited 130 states.
13 moves were used: ['U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D']
DFS took 0.0 seconds on map 3 and visited 72 states.
20 moves were used: ['U', 'L', 'U', 'U', 'R', 'D', 'D', 'D', 'U', 'L', 'D', 'U', 'R', 'D', 'D', 'U', 'U', 'L', 'D', 'D']
IDS took 0.02 seconds on map 3 and visited 73 states.
14 moves were used: ['U', 'L', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'U', 'U', 'L', 'D', 'D']
A* took 0.0 seconds on map 3 and visited 111 states.
13 moves were used: ['U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D']
Weighted A* took 0.0 seconds on map 3 and visited 47 states.
19 moves were used: ['U', 'L', 'D', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'U', 'L', 'D', 'D', 'U', 'U', 'R', 'D', 'D']


#### Map 4:

In [20]:
solve_all(4)

BFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
DFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
IDS took 0.06 seconds on map 4 and visited 2 states.
No Solution Found!
A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
Weighted A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!


#### Map 5:

In [21]:
solve_all(5)

BFS took 0.17 seconds on map 5 and visited 7896 states.
15 moves were used: ['U', 'L', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'U', 'R', 'U', 'L']
DFS took 0.08 seconds on map 5 and visited 4165 states.
187 moves were used: ['U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'R', 'U', 'U', 'R', 'R', 'D', 'D', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'D', 'L', 'U', 'L', 'L', 'D', 'D', 'R', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'R', 'R', 'D', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'R', 'U', 'U', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'R', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'U', 'U', 'R', 'U', 'U', 'L', 'D', 'U', 'L', 'D', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'R', 'U', 'L', 'U', 'L', 'D', 'U', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'L', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'U', 'U', 'L', 'U', 'R', 

#### Map 6:

In [22]:
solve_all(6)

BFS took 0.42 seconds on map 6 and visited 21150 states.
34 moves were used: ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'R']
DFS took 0.0 seconds on map 6 and visited 334 states.
114 moves were used: ['U', 'U', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'U', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'D', 'R']
IDS took 4.43 seconds on map 6 and visited 15332 states.
50 moves were used: ['U', 'U', 'U', 'U', 'U', 'L'

#### Map 7:

In [23]:
solve(7, "BFS")
solve(7, "A*")
solve(7, "Weighted A*")

BFS took 19.47 seconds on map 7 and visited 815887 states.
34 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R']
A* took 6.36 seconds on map 7 and visited 179699 states.
34 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R']
Weighted A* took 0.25 seconds on map 7 and visited 8674 states.
34 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R']


#### Map 8:

In [24]:
solve_all(8)

BFS took 0.21 seconds on map 8 and visited 10571 states.
14 moves were used: ['U', 'U', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'R', 'D', 'R']
DFS took 2.07 seconds on map 8 and visited 63298 states.
3960 moves were used: ['U', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'R', 'R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'R', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'L', 'L', 'U', 'U', 'L', 'U', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'L', 'D', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'R', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'R', 'R', 'U', 'U', 'U', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'L', 'U', 'U', 'L', 'L', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'R', 'U', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U

#### Map 9:

In [25]:
solve(9, "Weighted A*")

Weighted A* took 42.97 seconds on map 9 and visited 970754 states.
59 moves were used: ['R', 'U', 'R', 'R', 'R', 'L', 'D', 'D', 'L', 'U', 'L', 'U', 'R', 'R', 'U', 'R', 'D', 'R', 'D', 'R', 'D', 'L', 'D', 'L', 'U', 'R', 'U', 'U', 'U', 'R', 'D', 'R', 'D', 'L', 'L', 'U', 'U', 'L', 'L', 'U', 'U', 'U', 'L', 'L', 'L', 'U', 'R', 'R', 'U', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'D', 'R']


#### Map 10:

In [26]:
solve(10, "BFS")
solve(10, "DFS")
solve(10, "A*")
solve(10, "Weighted A*")

BFS took 6.11 seconds on map 10 and visited 255660 states.
46 moves were used: ['R', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'L', 'L', 'L', 'U', 'L', 'D', 'R', 'U', 'U', 'U', 'L', 'D', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'R', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'L']
DFS took 17.34 seconds on map 10 and visited 447639 states.
5663 moves were used: ['U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'U', 'R', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'L', 'L', 'U', 'U', 'R', 'U', 'U', 'L', 'D', 'L', 'D', 'D', 'R', 'U', 'L', 'D', 'D', 'D', 'R', 'R', 'U', 'R', 'U', 'U', 'R', 'R', 'D', 'D', 'D', 'L', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'L', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'U', 'R', 'U', 'U', 'L', 'U', 'U', 'L', 'L', 'D', 'L', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'L', 'D', 'D', 'R', 'U', 'D', 'R', 'R', 'U', 'U', 'U', 'R', 'R', 'D', 'D', 'L', 'U', 'L